In [9]:
import pandas as pd
import numpy as np
import os
import json
import sys 
import traceback

In [23]:
movies_path = os.path.join(os.path.join(os.getcwd(),'Datasets'),'movies_dataset')

metadata_path = os.path.join(movies_path,'movies_metadata.csv')
movies_genre_path = os.path.join(movies_path, 'movies.csv')
ratings_path = os.path.join(movies_path,'ratings.csv')
links_path = os.path.join(movies_path,'links.csv')

In [25]:
metadata = pd.read_csv(metadata_path)
print('metadata loaded')
ratings = pd.read_csv(ratings_path)
ratings = ratings.drop(columns='timestamp')
print('ratings loaded')
genres = pd.read_csv(movies_genre_path)
print('genre loaded')
links = pd.read_csv(links_path)
print('genre loaded')

metadata loaded
ratings loaded
genre loaded
genre loaded


In [32]:
metadata = metadata.drop(columns = ['adult', 'belongs_to_collection', 'budget', 'homepage', 'overview',
       'popularity', 'poster_path','release_date', 'original_title','revenue', 'runtime', 'status', 'tagline', 'video','vote_average', 'vote_count'])

metadata = metadata[metadata.id != '1997-08-20']
metadata = metadata[metadata.id != '2012-09-29']
metadata = metadata[metadata.id != '2014-01-01']
metadata = metadata.astype({'id':'float64'})

links.astype({'tmdbId':'float64'})

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
58093,193876,38566,78251.0
58094,193878,1754787,87558.0
58095,193880,5847740,422666.0
58096,193882,4453756,454439.0


In [50]:
merged = pd.merge(metadata, links, left_on=['id'], right_on=['tmdbId']).drop(columns=['id','imdb_id','imdbId','tmdbId'])
merged

,genres,original_language,production_companies,production_countries,spoken_languages,title,movieId
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,1
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,2
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,3
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,4
4,"[{'id': 35, 'name': 'Comedy'}]",en,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5
...,...,...,...,...,...,...,...
45507,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",fa,[],"[{'iso_3166_1': 'IR', 'name': 'Iran'}]","[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Subdue,176269
45508,"[{'id': 18, 'name': 'Drama'}]",tl,"[{'name': 'Sine Olivia', 'id': 19653}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]","[{'iso_639_1': 'tl', 'name': ''}]",Century of Birthing,176271
45509,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,"[{'name': 'American World Pictures', 'id': 6165}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Betrayal,176273
45510,[],en,"[{'name': 'Yermoliev', 'id': 88753}]","[{'iso_3166_1': 'RU', 'name': 'Russia'}]",[],Satan Triumphant,176275


In [51]:
metadata = merged
nrows = metadata.shape[0]

movies = {}
for index in range(nrows):

    # get the id of the movie
    try:
        id_row = int(metadata.iloc[index,:].movieId)
    except:
        print('-'*60)
        traceback.print_exc(file=sys.stdout)
        print('-'*60)
        continue


    # get the genres of the movie
    try:
         genres_row = eval(metadata.iloc[index,:].genres)
         genres_list = []
         for genres_dict in genres_row:
            id_ = genres_dict['id']
            genres_list.append(id_)
    except:
        print('-'*60)
        traceback.print_exc(file=sys.stdout)
        print('-'*60)
        continue

    # get original language
    try:
         original_row = metadata.iloc[index,:].original_language
    except:
        print('-'*60)
        traceback.print_exc(file=sys.stdout)
        print('-'*60)
        continue

    
    # get the production companies
    try:
        prod_comp_list = []
        prod_comp_row = eval(metadata.iloc[index,:].production_companies)
        for prod_comp_dict in prod_comp_row:
            id_ = prod_comp_dict['id']
            prod_comp_list.append(id_)
    except:
        print('-'*60)
        traceback.print_exc(file=sys.stdout)
        print('-'*60)
        continue

    # get the production countries
    try:
        prod_countries_list = []
        prod_countries_row = eval(metadata.iloc[index,:].production_countries)
        for prod_countries_dict in prod_countries_row:
            country = prod_countries_dict['iso_3166_1']
            prod_countries_list.append(country)
    except:
        print('-'*60)
        traceback.print_exc(file=sys.stdout)
        print('-'*60)
        continue

    # get the production countries
    try:
        spoken_languages_list = []
        spoken_languages_row = eval(metadata.iloc[index,:].spoken_languages)
        for spoken_languages_dict in spoken_languages_row:
            spoken_language = spoken_languages_dict['iso_639_1']
            spoken_languages_list.append(spoken_language)
    except:
        print('-'*60)
        traceback.print_exc(file=sys.stdout)
        print('-'*60)
        continue

    movies[id_row] = {'genres':genres_list,
                    'original_language':original_row,
                    'production_companies':prod_comp_list,
                    'production_countries': prod_countries_list,
                    'spoken_language':spoken_languages_list}
   

------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-51-e7e2ac94a4ce>", line 43, in <module>
    prod_comp_row = eval(metadata.iloc[index,:].production_companies)
TypeError: eval() arg 1 must be a string, bytes or code object
------------------------------------------------------------
------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-51-e7e2ac94a4ce>", line 43, in <module>
    prod_comp_row = eval(metadata.iloc[index,:].production_companies)
TypeError: eval() arg 1 must be a string, bytes or code object
------------------------------------------------------------
------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-51-e7e2ac94a4ce>", line 43, in <module>
    prod_comp_row = eval(metadata.iloc[index,:].production_companies)
TypeError: eval() arg 1 must be a string, bytes or code object
--

In [52]:
len(movies)

45447

In [53]:
with open('./movies_metadata.txt','w') as fw:
    json.dump(movies, fw)

In [8]:
with  open('./movies_metadata.txt','r') as fr:
    data = json.load(fr)
print(len(data))

45430


In [16]:
def filter_users(ratings):
    # count occurrences of same userId
    print(ratings.shape)
    ids = ratings.userId
    counts = ids.value_counts()

    # select ids to keep (users with #ratings >= 5)
    to_keep = counts[counts.values >= 50].index
    mask = [(x in to_keep) for x in ratings.userId]
    ratings_5 = ratings[mask]
    return ratings_5

In [17]:
def filter_movies(ratings):
    # count occurrences of same movieId
    print(ratings.shape)
    ids = ratings.movieId
    counts = ids.value_counts()

    # select ids to keep (movies with #ratings >= 20)
    to_keep = counts[counts.values >= 50].index
    mask = [(x in to_keep) for x in ratings.movieId]
    ratings_20 = ratings[mask]
    return ratings_20


In [18]:
def filter(ratings):
    num_rows = ratings.shape[0]

    tmp = ratings
    while True:
        tmp_5 = filter_users(tmp)
        if tmp_5.shape[0] == tmp.shape[0]:
            print('convergence has been reached')
            print('users now are {}'.format(len(tmp_5.userId.unique())))
            print('movies are now {}'.format(len(tmp_20.movieId.unique())))
            return tmp_5
        else:
            print('users filtered')
            print('users now are {}'.format(len(tmp_5.userId.unique())))
            tmp = tmp_5
        tmp_20 = filter_movies(tmp)
        if tmp_20.shape[0] == tmp.shape[0]:
            print('convergence has been reached')
            print('users now are {}'.format(len(tmp_5.userId.unique())))
            print('movies are now {}'.format(len(tmp_20.movieId.unique())))
            return tmp_20
        else:
            print('movies filtered')
            print('movies are now {}'.format(len(tmp_20.movieId.unique())))
            tmp = tmp_20



In [19]:
final_ratings = filter(ratings)

(26024289, 3)
users filtered
users now are 103839
(22879277, 3)
movies filtered
movies are now 12341
(22610090, 3)
users filtered
users now are 103652
(22601367, 3)
movies filtered
movies are now 12333
(22600977, 3)
users filtered
users now are 103651
(22600928, 3)
convergence has been reached
users now are 103651
movies are now 12333


In [13]:
len(final_ratings.movieId.unique())

12333

In [9]:
def build_user_based_dictionary(unique_users, ratings):
    print('\nunique users in this batch are from {} to {}'.format(unique_users[0], unique_users[-1]))
    print('\nthe shape of ratings is {}'.format(ratings.shape))
    sparse_user_based = {}

    for user in unique_users:
        user_ratings = {}
        user_subtable = ratings[ratings.userId == user]
        for index,row in user_subtable.iterrows():
            user_ratings[str(row['movieId'])] = str(row['rating'])
        sparse_user_based[str(user)] = user_ratings
        if user%1000 in [0,1]:
            print('user is {}'.format(user))
    return sparse_user_based

In [10]:
def build_item_based_dictionary(unique_movies, ratings):
    print('\nunique movies in this batch are from {} to {}'.format(unique_movies[0], unique_movies[-1]))
    print('\nthe shape of ratings is {}'.format(ratings.shape))
    sparse_item_based = {}

    for movie in unique_movies:
        movie_ratings = {}
        movie_subtable = ratings[ratings.movieId == movie]
        for index,row in movie_subtable.iterrows():
            movie_ratings[str(row['userId'])] = str(row['rating'])
        sparse_item_based[str(movie)] = movie_ratings
        if movie%500 in [0,1]:
            print('movie is {}'.format(movie))
    return sparse_item_based

In [11]:
from threading import Thread

class CreateUtilityThread(Thread):
    def __init__(self, name = None, unique_users = None, unique_items = None, ratings = None, job = None):
        Thread.__init__(self)
        self.name = name
        self.unique_users = unique_users
        self.unique_items = unique_items
        self.ratings = ratings
        self.job = job
        self._return = None

    def run(self):
        print(self.name)
        if self.name is not None:
            if self.job == 'item_based':
                self._return = build_item_based_dictionary(self.unique_items, self.ratings)
            elif self.job == 'user_based':
                self._return = build_user_based_dictionary(self.unique_users, self.ratings)
            else:
                print('Error, you have to specify a job')

    def join(self):
        Thread.join(self)
        if self._return is not None:
            return self._return
        else:
            print('Error using threading')

In [21]:
def split_list(l):
    n = len(l) // 2          # length of smaller half
    
    l1 = l[:n]
    l2 = l[n:]

    return l1, l2

In [22]:
unique_users = final_ratings.userId.unique()

# split the dictionary in 2
r1, r2 = split_list(unique_users)

# split the two dictionaries in 4
r11, r12 = split_list(r1)
r21, r22 = split_list(r2)

# split the 4 dictionaries in 8
r1, r2 = split_list(r11)
r3, r4 = split_list(r12)
r5, r6 = split_list(r21)
r7, r8 = split_list(r22)

thread1 = CreateUtilityThread('thread-1', unique_users = r1, ratings = final_ratings, job = 'user_based')
thread2 = CreateUtilityThread('thread-2', unique_users = r2, ratings = final_ratings, job = 'user_based')
thread3 = CreateUtilityThread('thread-3', unique_users = r3, ratings = final_ratings, job = 'user_based')
thread4 = CreateUtilityThread('thread-4', unique_users = r4, ratings = final_ratings, job = 'user_based')
thread5 = CreateUtilityThread('thread-5', unique_users = r5, ratings = final_ratings, job = 'user_based')
thread6 = CreateUtilityThread('thread-6', unique_users = r6, ratings = final_ratings, job = 'user_based')
thread7 = CreateUtilityThread('thread-7', unique_users = r7, ratings = final_ratings, job = 'user_based')
thread8 = CreateUtilityThread('thread-8', unique_users = r8, ratings = final_ratings, job = 'user_based')

threads = [thread1,thread2,thread3,thread4,thread5,thread6,thread7,thread8]

[thread.start() for thread in threads]
print('============= all threads started =============')

dictionaries = [thread.join() for thread in threads]
print('============= all threads joined =============')

thread-1thread-2

unique users in this batch are from 34010 to 67769

the shape of ratings is (25709727, 3)


unique users in this batch are from 1 to 34009

the shape of ratings is (25709727, 3)thread-3


unique users in this batch are from 67770 to 101724

the shape of ratings is (25709727, 3)
thread-4

unique users in this batch are from 101725 to 135636

the shape of ratings is (25709727, 3)
thread-5

unique users in this batch are from 135637 to 169389

the shape of ratings is (25709727, 3)
thread-6thread-7

unique users in this batch are from 203208 to 236978

the shape of ratings is (25709727, 3)


unique users in this batch are from 169394 to 203207

the shape of ratings is (25709727, 3)
thread-8============= all threads started =============


unique users in this batch are from 236979 to 270896

the shape of ratings is (25709727, 3)
user is 1
user is 237000
user is 237001
user is 68000
user is 68001
user is 102000
user is 102001
user is 136000
user is 136001
user is 170001
us

In [23]:
utility_matrix_user_based = {**dictionaries[0],**dictionaries[1],**dictionaries[2],**dictionaries[3],
                            **dictionaries[4],**dictionaries[5],**dictionaries[6],**dictionaries[7]}
print('the number of unique_users is {} and the sanity check is {}'.format(len(unique_users),len(utility_matrix_user_based) == len(unique_users)))

the number of unique_users is 233714 and the sanity check is True


In [25]:
with open('./utility_matrix_user_based.txt','w') as fw:
    json.dump(utility_matrix_user_based, fw)

TypeError: keys must be a string

In [ ]:
with  open('./utility_matrix_user_based.txt','r') as fr:
    data = json.load(fr)
print(len(data), len(data['2']))

In [ ]:
unique_movies = np.sort(final_ratings.movieId.unique())

# split the dictionary in 2
r1, r2 = split_list(unique_movies)

# split the two dictionaries in 4
r11, r12 = split_list(r1)
r21, r22 = split_list(r2)

# split the 4 dictionaries in 8
r1, r2 = split_list(r11)
r3, r4 = split_list(r12)
r5, r6 = split_list(r21)
r7, r8 = split_list(r22)

thread1 = CreateUtilityThread('thread-1', unique_items = r1, ratings = final_ratings, job = 'item_based')
thread2 = CreateUtilityThread('thread-2', unique_items = r2, ratings = final_ratings, job = 'item_based')
thread3 = CreateUtilityThread('thread-3', unique_items = r3, ratings = final_ratings, job = 'item_based')
thread4 = CreateUtilityThread('thread-4', unique_items = r4, ratings = final_ratings, job = 'item_based')
thread5 = CreateUtilityThread('thread-5', unique_items = r5, ratings = final_ratings, job = 'item_based')
thread6 = CreateUtilityThread('thread-6', unique_items = r6, ratings = final_ratings, job = 'item_based')
thread7 = CreateUtilityThread('thread-7', unique_items = r7, ratings = final_ratings, job = 'item_based')
thread8 = CreateUtilityThread('thread-8', unique_items = r8, ratings = final_ratings, job = 'item_based')

threads = [thread1,thread2,thread3,thread4,thread5,thread6,thread7,thread8]

[thread.start() for thread in threads]
print('============= all threads started =============')

dictionaries = [thread.join() for thread in threads]
print('============= all threads joined =============')

In [ ]:
utility_matrix_item_based = {**dictionaries[0],**dictionaries[1],**dictionaries[2],**dictionaries[3],
                            **dictionaries[4],**dictionaries[5],**dictionaries[6],**dictionaries[7]}
print('the number of unique_items is {} and the sanity check is {}'.format(len(unique_movies),len(utility_matrix_item_based) == len(unique_movies)))

In [ ]:
with open('./utility_matrix_item_based.txt','w') as fw:
    json.dump(utility_matrix_item_based, fw)

In [2]:
with  open('./utility_matrix_item_based.txt','r') as fr:
    data = json.load(fr)
print(len(data))

16770
